In [1]:
import pandas as pd
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

In [ ]:
dfTest = pd.read_csv('../data/test.csv',usecols=['srch_id','prop_id','prop_starrating', 'prop_review_score', 'prop_brand_bool',
           'prop_location_score1', 'prop_location_score2',
           'prop_log_historical_price','srch_destination_id','orig_destination_distance','price_usd'])
dfTrain = pd.read_csv('../data/train.csv',usecols=['srch_id','prop_id','prop_starrating', 'prop_review_score', 'prop_brand_bool',
           'prop_location_score1', 'prop_location_score2',
           'prop_log_historical_price','srch_destination_id','booking_bool','orig_destination_distance','price_usd'])

In [2]:
path = '/Users/cathytol/Documents/DMT/dmt-data/'

inputFile = path + "training_set_VU_DM.csv"
dfTrain = pd.read_csv(inputFile, sep = ',', usecols=['srch_id','prop_id','prop_starrating', 'prop_review_score', 'prop_brand_bool',
           'prop_location_score1', 'prop_location_score2',
           'prop_log_historical_price','srch_destination_id','booking_bool','orig_destination_distance','price_usd'])
print(len(dfTrain))

inputFile = path + "test_set_VU_DM.csv"
dfTest = pd.read_csv(inputFile, sep = ',', usecols=['srch_id','prop_id','prop_starrating', 'prop_review_score', 'prop_brand_bool',
           'prop_location_score1', 'prop_location_score2',
           'prop_log_historical_price','srch_destination_id','orig_destination_distance','price_usd'])
print(len(dfTest))

together = pd.concat([dfTest,dfTrain],axis=0)
print(len(together))

4958347
4959183


/Users/cathytol/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  from ipykernel import kernelapp as app


9917530


In [ ]:
together = pd.concat([dfTest,dfTrain],axis=0)

In [3]:
meanTestDist = dfTest[['srch_id','orig_destination_distance']].groupby(['srch_id']).mean()
meanTestDist = meanTestDist.rename(columns={'orig_destination_distance':'mean_distance'})
dfTest = dfTest.merge(right=meanTestDist,how="left",on="srch_id",validate="many_to_one")
diff_distance_orig_Test = (dfTest['orig_destination_distance'] - dfTest['mean_distance'])/dfTest['mean_distance']


In [4]:
meanTrainDist = dfTrain[['srch_id','orig_destination_distance']].groupby(['srch_id']).mean()
meanTrainDist = meanTrainDist.rename(columns={'orig_destination_distance':'mean_distance'})
dfTrain = dfTrain.merge(right=meanTrainDist,how="left",on="srch_id",validate="many_to_one")
diff_distance_orig_Train = (dfTrain['orig_destination_distance'] - dfTrain['mean_distance'])/dfTrain['mean_distance']


In [5]:
meanTestPricePerSRCHID = dfTest[['srch_id','price_usd']].groupby(['srch_id']).mean()
meanTestPricePerSRCHID = meanTestPricePerSRCHID.rename(columns={'price_usd':'mean_price_usd'})
dfTest = dfTest.merge(right=meanTestPricePerSRCHID,how="left",on="srch_id",validate="many_to_one")
diff_mean_price_test = (dfTest['price_usd'] - dfTest['mean_price_usd'])/dfTest['mean_price_usd']


In [6]:
meanTrainPricePerSRCHID = dfTrain[['srch_id','price_usd']].groupby(['srch_id']).mean()
meanTrainPricePerSRCHID = meanTrainPricePerSRCHID.rename(columns={'price_usd':'mean_price_usd'})
dfTrain = dfTrain.merge(right=meanTestPricePerSRCHID,how="left",on="srch_id",validate="many_to_one")
diff_mean_price_train = (dfTrain['price_usd'] - dfTrain['mean_price_usd'])/dfTrain['mean_price_usd']


In [7]:
columnsToAvgPerPropID = ['prop_starrating', 'prop_review_score', 'prop_brand_bool',
           'prop_location_score1', 'prop_location_score2',
           'prop_log_historical_price']
groupedMean = together.groupby('prop_id')[columnsToAvgPerPropID].mean()
groupedMean = groupedMean.add_prefix("mean_").reset_index()
groupedMean.to_csv("meansTotal.csv",index=False)

In [8]:
groupedPropDest = dfTrain.groupby(['srch_destination_id','prop_id']).mean().reset_index()
groupedPropDest = groupedPropDest.rename(columns = {"booking_bool": "booking_prop_dest_mean"})

In [9]:
groupedPropDest.head()

srch_destination_id  prop_id   srch_id  prop_starrating  prop_review_score  \
0                    2      985  246762.0              3.0                4.0   
1                    2     1974  200093.0              3.0                4.0   
2                    2     3003  200093.0              5.0                4.5   
3                    2    10729  223427.5              0.0                4.0   
4                    2    14413  246762.0              3.0                2.0   

   prop_brand_bool  prop_location_score1  prop_location_score2  \
0              1.0                  3.18                   NaN   
1              1.0                  3.40                0.0220   
2              1.0                  3.64                0.0639   
3              0.0                  3.58                0.0556   
4              0.0                  3.58                0.0998   

   prop_log_historical_price  price_usd  orig_destination_distance  \
0                      0.000    110.360                        NaN   
1                      5.110    188.100                        NaN   
2                      5.370    227.630                        NaN   
3                      2.695    201.665                        NaN   
4                      0.000     88.520                        NaN   

   booking_prop_dest_mean  mean_distance  mean_price_usd  
0                     0.0            NaN       89.909000  
1                     0.0            NaN       68.941176  
2                     1.0            NaN       68.941176  
3                     0.0            NaN       79.425088  
4                     1.0            NaN       89.909000

In [10]:
groupedPropTotal = dfTrain.groupby(['prop_id']).mean().reset_index()
groupedPropTotal = groupedPropTotal.rename(columns = {"booking_bool": "booking_prop_total_mean"})

In [11]:
def joinPropDest(df):
    df = df.merge(right=groupedPropDest[['srch_destination_id','prop_id','booking_prop_dest_mean']],how="left",on=['srch_destination_id','prop_id'],validate="many_to_one")
    return df
def joinPropDestTotal(df):
    df = df.merge(right=groupedPropTotal[['prop_id','booking_prop_total_mean']],how="left",on=['prop_id'],validate="many_to_one")
    return df

In [12]:
def joinDiffPriceTest(df):
    df['diff_price_usd']=diff_mean_price_test
    return df
def joinDiffPriceTrain(df):
    df['diff_price_usd']=diff_mean_price_train
    return df

In [13]:
def joinDistanceTest(df):
    df['diff_distance_orig']=diff_distance_orig_Test
    return df
def joinDistanceTrain(df):
    df['diff_distance_orig']=diff_distance_orig_Train
    return df

In [14]:
def joinMean(df):
    df = df.merge(right=meanDF,how="left",on="prop_id",validate="many_to_one")
    return df

In [15]:
meanDF= pd.read_csv("meansTotal.csv")

In [16]:
mms = MinMaxScaler()
data = meanDF.drop(['prop_id','mean_prop_review_score','mean_prop_location_score2'],axis=1)
mms.fit(data)
data_transformed = mms.transform(data)
km = KMeans(n_clusters=6)
km = km.fit(data_transformed)

/Users/cathytol/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [17]:
dummiesCluster = pd.get_dummies(km.labels_,prefix = "cluster_")

In [18]:
meanDF = pd.concat([meanDF,dummiesCluster],axis=1)

In [26]:
path = '/Users/cathytol/Documents/DMT/dmt-data/'

#inputFile = path + "test_set_VU_DM.csv"
inputFile = path + "training_set_VU_DM.csv"
df = pd.read_csv(inputFile, sep = ',')
print(len(df))



4958347


In [ ]:
#df = pd.read_csv('../data/test.csv')

In [20]:
df = joinMean(df)

In [21]:
df = joinPropDest(df)
df = joinPropDestTotal(df)
df = joinDiffPriceTest(df)

In [22]:
df = joinDistanceTest(df)

In [23]:
df.head()

srch_id            date_time  site_id  visitor_location_country_id  \
0        1  2013-02-02 15:27:40       24                          216   
1        1  2013-02-02 15:27:40       24                          216   
2        1  2013-02-02 15:27:40       24                          216   
3        1  2013-02-02 15:27:40       24                          216   
4        1  2013-02-02 15:27:40       24                          216   

   visitor_hist_starrating  visitor_hist_adr_usd  prop_country_id  prop_id  \
0                      NaN                   NaN              219     3180   
1                      NaN                   NaN              219     5543   
2                      NaN                   NaN              219    14142   
3                      NaN                   NaN              219    22393   
4                      NaN                   NaN              219    24194   

   prop_starrating  prop_review_score  ...  cluster__0  cluster__1  \
0                3                4.5  ...           0           0   
1                3                4.5  ...           0           0   
2                2                3.5  ...           0           1   
3                3                4.5  ...           0           0   
4                3                4.5  ...           0           0   

   cluster__2  cluster__3  cluster__4  cluster__5  booking_prop_dest_mean  \
0           0           0           0           1                0.077778   
1           0           0           0           1                0.142857   
2           0           0           0           0                0.000000   
3           0           0           0           1                0.026316   
4           0           0           0           1                0.010870   

   booking_prop_total_mean  diff_price_usd  diff_distance_orig  
0                 0.043011        0.268703                 NaN  
1                 0.053279        0.258042                 NaN  
2                 0.013423       -0.477593                 NaN  
3                 0.013793        0.524576                 NaN  
4                 0.014634       -0.157752                 NaN  

[5 rows x 66 columns]

In [25]:
df.to_csv('test_final_new.csv', index = False)

In [ ]:
#df.to_csv('../data/testWithMean.csv',index=False)

In [27]:
#df = pd.read_csv('../data/train.csv')
df = joinMean(df)
df = joinPropDest(df)
df = joinPropDestTotal(df)
df = joinDistanceTrain(df)
df = joinDiffPriceTrain(df)
df.to_csv('train_final_new.csv', index = False)
#df.to_csv('../data/trainWithMean.csv',index=False)

Split train in train and test

In [ ]:
#df = shuffle(df,random_state = 10)
#df=pd.read_csv('../data/trainWithMean.csv')
df=pd.read_csv('train_final_new.csv')

In [28]:
n = 20
numOfRowsTest = (int(len(df)*(n/100)))
X_test = df.tail(numOfRowsTest)
df.drop(df.tail(numOfRowsTest).index,inplace=True)

In [29]:
X_test.to_csv('20PercentTestWithMeanNoShuffle.csv',index=False)
df.to_csv('80PercentTrainWithMeanNoShuffle.csv',index=False)